<a href="https://colab.research.google.com/github/kaushik3012/EvoDiff_active_site/blob/main/EvoDiff_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("fireprot_homologue_free.csv", index_col=0)
df

,WT_name,ddG,wild_type,mutation,pdb_position,pdb_sequence,aa_seq,mut_type
0,5DFR.pdb,1.575000,V,A,39,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPAIMGRHT...,V39A
1,5DFR.pdb,1.700000,V,H,74,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,V74H
2,5DFR.pdb,1.633333,V,I,74,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,V74I
3,5DFR.pdb,1.010000,V,I,87,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,V87I
4,5DFR.pdb,1.000000,I,A,114,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,I114A
...,...,...,...,...,...,...,...,...
2573,1ISP.pdb,0.090000,Y,N,159,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,Y159N
2574,1ISP.pdb,-0.360000,N,T,172,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,N172T
2575,2LZM.pdb,1.710000,W,Y,137,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,W137Y
2576,2AFG.pdb,-0.070000,F,W,75,KPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVY...,KPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVY...,F75W


In [ ]:
df['pdb_position'] = df['pdb_position'].astype(int)
df['pdb_sequence'] = df['pdb_sequence'].astype(str)


In [ ]:
x = df.iloc[0]
x['pdb_sequence'][:x['pdb_position']] + '#' + x['pdb_sequence'][x['pdb_position'] + 1:]

'MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKP#IMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR'

In [ ]:
df['masked_seq'] = df.apply(lambda x: x['pdb_sequence'][:x['pdb_position']] + '#' + x['pdb_sequence'][x['pdb_position'] + 1:], axis=1)
df

,WT_name,ddG,wild_type,mutation,pdb_position,pdb_sequence,aa_seq,mut_type,masked_seq
0,5DFR.pdb,1.575000,V,A,39,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPAIMGRHT...,V39A,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKP#IMGRHT...
1,5DFR.pdb,1.700000,V,H,74,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,V74H,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...
2,5DFR.pdb,1.633333,V,I,74,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,V74I,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...
3,5DFR.pdb,1.010000,V,I,87,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,V87I,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...
4,5DFR.pdb,1.000000,I,A,114,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,I114A,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...
...,...,...,...,...,...,...,...,...,...
2573,1ISP.pdb,0.090000,Y,N,159,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,Y159N,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...
2574,1ISP.pdb,-0.360000,N,T,172,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...,N172T,EHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGT...
2575,2LZM.pdb,1.710000,W,Y,137,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,W137Y,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...
2576,2AFG.pdb,-0.070000,F,W,75,KPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVY...,KPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVY...,F75W,KPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVY...


In [ ]:
df_group = df.groupby("WT_name")

In [ ]:
df_sub = df.groupby("WT_name").filter(lambda x: any(x.count()>1))

In [ ]:
df_sub = df_sub.groupby("WT_name").sample(2, replace=False)

In [ ]:
df_sub

,WT_name,ddG,wild_type,mutation,pdb_position,pdb_sequence,aa_seq,mut_type,masked_seq
1031,1A23.pdb,-3.966667,H,Y,31,AQYEDGKQYTTLEKPVAGAPQVLEFFSFFCPHCYQFEEVLHISDNV...,AQYEDGKQYTTLEKPVAGAPQVLEFFSFFCPYCYQFEEVLHISDNV...,H31Y,AQYEDGKQYTTLEKPVAGAPQVLEFFSFFCP#CYQFEEVLHISDNV...
1720,1A23.pdb,-4.400000,H,S,31,AQYEDGKQYTTLEKPVAGAPQVLEFFSFFCPHCYQFEEVLHISDNV...,AQYEDGKQYTTLEKPVAGAPQVLEFFSFFCPSCYQFEEVLHISDNV...,H31S,AQYEDGKQYTTLEKPVAGAPQVLEFFSFFCP#CYQFEEVLHISDNV...
460,1A2P.pdb,-0.503333,S,A,25,VINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPG...,VINTFDGVADYLQTYHKLPDNYITKAEAQALGWVASKGNLADVAPG...,S25A,VINTFDGVADYLQTYHKLPDNYITK#EAQALGWVASKGNLADVAPG...
140,1A2P.pdb,0.315000,R,A,107,VINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPG...,VINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPG...,R107A,VINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPG...
1816,1A5E.pdb,-0.170000,W,D,14,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...,MEPAAGSSMEPSADDLATAAARGRVEEVRALLEAGALPNAPNSYGR...,W14D,MEPAAGSSMEPSAD#LATAAARGRVEEVRALLEAGALPNAPNSYGR...
...,...,...,...,...,...,...,...,...,...
1563,4LYZ.pdb,-0.990000,S,T,90,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,S90T,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...
1714,5DFR.pdb,1.100000,T,A,34,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNALDKPVIMGRHT...,T34A,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRN#LDKPVIMGRHT...
9,5DFR.pdb,2.292500,I,L,154,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...,I154L,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...
1596,5PTI.pdb,-0.200000,D,A,2,RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFK...,RPAFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFK...,D2A,RP#FCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFK...


In [ ]:
df_train = pd.DataFrame(columns=['prompt', 'chosen', 'rejected'])
for row in df_sub.groupby("WT_name"):
    if row[1].iloc[0].ddG < row[1].iloc[1].ddG:
        df_train.loc[-1] = [row[1].iloc[0]['masked_seq'], row[1].iloc[0]['aa_seq'], row[1].iloc[1]['aa_seq']]  # adding a row
    else:
        df_train.loc[-1] = [row[1].iloc[0]['masked_seq'], row[1].iloc[1]['aa_seq'], row[1].iloc[0]['aa_seq']]  # adding a row
    df_train.index = df_train.index + 1  # shifting index
    df_train = df_train.sort_index()  # sorting by index

In [ ]:
df_train

,prompt,chosen,rejected
0,RP#FCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFK...,RPAFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFK...,RPDFCLEPPATGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFK...
1,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRN#LDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNALDKPVIMGRHT...,MISLIAALAVDRVIGXXXXXPWNLPADLAWFKRNTLDKPVIMGRHT...
2,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...
3,DDFAKLEEQFDAKLGIFALDTGTNRTVXAYRPDERFAFASTIKALT...,DDFAKLEEQFDAKLGIFALDTGTNRTVXAYRPDERFAFASTIKALT...,DDFAKLEEQFDAKLGIFALDTGTNRTVXAYRPDERFAFASTIKALT...
4,EDPEVLFKNKGC#ACHAIDTKMVGPAYKDVAAKFAGQAGAEAELAQ...,EDPEVLAKNKGCVACHAIDTKMVGPAYKDVAAKFAGQAGAEAELAQ...,EDPEVLFKNKGCMACHAIDTKMVGPAYKDVAAKFAGQAGAEAELAQ...
...,...,...,...
75,GLGGYMLGSAMSRPMIHFGNDWEDRYYRENMYRYPNQVYYRPVDQY...,GLGGYMLGSAMSRPMIHFGNDWEDRYYRENMYRYPNQVYYRPVDQY...,GLGGYMLGSAMSRPMIHFGNDWEDRYYRENMYRYPNQVYYRPVDQY...
76,NIAEAVQQLNHTIVNAAHELHE#LGLPTPDEALNLLTEQANAFKTK...,NIAEAVQQLNHTIVNAAHELHEALGLPTPDEALNLLTEQANAFKTK...,NIAEAVQQKNHTIVNAAHELHETLGLPTPDEALNLLTEQANAFKTK...
77,MEPAAGSSMEPSAD#LATAAARGRVEEVRALLEAGALPNAPNSYGR...,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...,MEPAAGSSMEPSADDLATAAARGRVEEVRALLEAGALPNAPNSYGR...
78,VINTFDGVADYLQTYHKLPDNYITK#EAQALGWVASKGNLADVAPG...,VINTFDGVADYLQTYHKLPDNYITKAEAQALGWVASKGNLADVAPG...,VINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPG...


In [ ]:
import argparse
import random
import numpy as np
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM

from tqdm import tqdm

def seed_everything(seed=2003):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def calculate_DPO_loss(model_prefered_logprob, model_disprefered_logprob,
                       ref_prefered_logprob, ref_disprefered_logprob,
                       beta=0.5):

    prefered_relative_logprob = model_prefered_logprob - ref_prefered_logprob
    disprefered_relative_logprob = model_disprefered_logprob - ref_disprefered_logprob

    reward_accuracies = (prefered_relative_logprob > disprefered_relative_logprob).float().mean(dim=-1)
    reward_margins = (prefered_relative_logprob - disprefered_relative_logprob).mean(dim=-1)

    loss = -F.logsigmoid(beta * (prefered_relative_logprob - disprefered_relative_logprob)).mean(dim=-1)

    return loss, prefered_relative_logprob.mean(dim=-1), disprefered_relative_logprob.mean(dim=-1), reward_accuracies, reward_margins

def get_log_prob(logits, labels):
    log_probs = F.log_softmax(logits, dim=-1)
    return torch.gather(log_probs, -1, labels.unsqueeze(-1)).squeeze(-1).mean(-1)

def collate_fn(batch, tokenizer, max_length, device):
    prompts = ['Instruct: ' + item['prompt'] + '\n' for item in batch]
    chosen_responses = ['Output: ' + item['chosen'] for item in batch]
    rejected_responses = ['Output: ' + item['rejected'] for item in batch]

    prompt_ids = tokenizer.batch_encode_plus(prompts, padding=True, return_tensors="pt", max_length=max_length, truncation=True)['input_ids'].to(device)
    prefered_ids = tokenizer.batch_encode_plus(chosen_responses, padding=True, return_tensors="pt", max_length=max_length, truncation=True)['input_ids'].to(device)
    disprefered_ids = tokenizer.batch_encode_plus(rejected_responses, padding=True, return_tensors="pt", max_length=max_length, truncation=True)['input_ids'].to(device)

    prompt_prefered_ids = torch.cat([prompt_ids, prefered_ids], dim=-1)
    prompt_disprefered_ids = torch.cat([prompt_ids, disprefered_ids], dim=-1)

    prompt_prefered_mask = torch.cat([torch.ones_like(prompt_ids), torch.zeros_like(prefered_ids)], dim=-1)
    prompt_disprefered_mask = torch.cat([torch.ones_like(prompt_ids), torch.zeros_like(disprefered_ids)], dim=-1)

    return {'prompt_prefered_ids': prompt_prefered_ids,
            'prompt_disprefered_ids': prompt_disprefered_ids,
            'prompt_prefered_mask': prompt_prefered_mask,
            'prompt_disprefered_mask': prompt_disprefered_mask}

def train(model, ref_model, tokenizer, optimizer, train_dataloader, epochs=1, beta=0.1):
    model.train()
    ref_model.eval()

    for epoch in range(epochs):
        for batch in tqdm(train_dataloader):
            optimizer.zero_grad()

            prompt_prefered_ids = batch['prompt_prefered_ids']
            prompt_disprefered_ids = batch['prompt_disprefered_ids']
            prompt_prefered_mask = batch['prompt_prefered_mask']
            prompt_disprefered_mask = batch['prompt_disprefered_mask']

            model_prefered_log_prob = get_log_prob(model(prompt_prefered_ids, attention_mask=prompt_prefered_mask).logits, prompt_prefered_ids)
            model_disprefered_log_prob = get_log_prob(model(prompt_disprefered_ids, attention_mask=prompt_disprefered_mask).logits, prompt_disprefered_ids)

            ref_prefered_log_prob = get_log_prob(ref_model(prompt_prefered_ids, attention_mask=prompt_prefered_mask).logits, prompt_prefered_ids)
            ref_disprefered_log_prob = get_log_prob(ref_model(prompt_disprefered_ids, attention_mask=prompt_disprefered_mask).logits, prompt_disprefered_ids)

            loss, prefered_relative_logprob, disprefered_relative_logprob, reward_accuracies, reward_margins = calculate_DPO_loss(model_prefered_log_prob, model_disprefered_log_prob,
                                          ref_prefered_log_prob, ref_disprefered_log_prob,
                                          beta=beta)

            loss.backward()
            optimizer.step()

            print({'loss': loss.item(),
                       'prefered_relative_logprob': prefered_relative_logprob,
                       'disprefered_relative_logprob': disprefered_relative_logprob,
                       'reward_accuracy': reward_accuracies,
                       'reward_margin': reward_margins})



In [ ]:

# def main():
#     parser = argparse.ArgumentParser()

#     parser.add_argument("--epochs", type=int, default=1)
#     parser.add_argument("--beta", type=float, default=0.1)
#     parser.add_argument("--batch_size", type=int, default=4)
#     parser.add_argument("--max_length", type=int, default=512)
#     parser.add_argument("--lr", type=float, default=1e-6)
#     parser.add_argument("--seed", type=int, default=2003)
#     parser.add_argument("--model_name", type=str, default="microsoft/phi-2")
#     parser.add_argument("--dataset_name", type=str, default="jondurbin/truthy-dpo-v0.1")
#     parser.add_argument("--wandb_project", type=str, default="truthy-dpo")

#     args = parser.parse_args()

seed_everything(2003)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(args.model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(args.model_name).to(device)
ref_model = AutoModelForCausalLM.from_pretrained(args.model_name).to(device)

optimizer = AdamW(model.parameters(), lr=args.lr)

dataset = load_dataset(args.dataset_name, split="train")
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, shuffle=True, collate_fn=partial(collate_fn, tokenizer=tokenizer, max_length=args.max_length, device=device))

train(model, ref_model, tokenizer, optimizer, train_dataloader, epochs=args.epochs, beta=args.beta)

model.save_pretrained("model-DPO.pt")